# Issue 1: Export Roslagsleden Trail Geojson version 2

* https://github.com/salgo60/Roslagsleden/issues/1
* [this notebook](https://github.com/salgo60/Roslagsleden/blob/main/notebook/Issue%201%20Export%20Roslagen%20Trail%20Geojson.ipynb)

## 🧭 Export P625 Point Data for SAT Trail Sections from Wikidata

This notebook cell performs the following:

1. Runs a SPARQL query to retrieve SAT trail sections (`wd:Q10657105`) with:
   - `wdt:P625` coordinate location
   - `wdt:P18` image (optional)
   - `wdt:P856` official website (optional)
   - `wdt:P402` OSM relation ID
   - Item label and Wikidata URI

2. Creates a GeoJSON file with:
   - **Point geometry** from P625
   - **Popup info** (name, image, website, links to Wikidata/OSM) in a field called `UmapPop`

You can import the result into **uMap** and use `UmapPop` as the popup template field.

---

### ✅ Python code:



In [2]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-07-27 19:07


In [3]:
import requests
import json
import datetime
from shapely.geometry import Point, mapping
from geojson import Feature, FeatureCollection

# SPARQL query to get SAT trail sections with P625 + metadata
sparql_query = """
SELECT ?item ?itemLabel ?coord ?image ?website ?OSMrel WHERE {
  ?item wdt:P361 wd:Q10657105;
        wdt:P31 wd:Q2143825.
  OPTIONAL { ?item wdt:P625 ?coord. }
  OPTIONAL { ?item wdt:P18 ?image. }
  OPTIONAL { ?item wdt:P856 ?website. }
  OPTIONAL { ?item wdt:P402 ?OSMrel. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY (?itemLabel)
"""

# Run SPARQL query
def run_sparql(query):
    url = "https://query.wikidata.org/sparql"
    headers = {'Accept': 'application/sparql-results+json'}
    r = requests.get(url, params={'query': query}, headers=headers)
    data = r.json()
    results = []
    for row in data["results"]["bindings"]:
        item_uri = row["item"]["value"]
        label = row.get("itemLabel", {}).get("value")
        coord = row.get("coord", {}).get("value")
        image = row.get("image", {}).get("value")
        website = row.get("website", {}).get("value")
        rel_id = row.get("OSMrel", {}).get("value")
        if coord:
            lat, lon = map(float, coord.replace("Point(", "").replace(")", "").split())
            results.append({
                "label": label,
                "coord": (lon, lat),  # GeoJSON uses (lon, lat)
                "image": image,
                "website": website,
                "osm_rel": rel_id,
                "wikidata": item_uri
            })
    return results

# Build GeoJSON with point geometry and rich popup
def build_geojson_points(sections):
    features = []
    for s in sections:
        props = {
            "name": s["label"],
            "wikidata": s["wikidata"],
            "osm_relation": f"https://www.openstreetmap.org/relation/{s['osm_rel']}" if s.get("osm_rel") else None,
            "website": s.get("website"),
            "image": s.get("image")
        }

        # Rich popup for uMap
        umap_html = f"<b>{s['label']}</b><br>"
        if s.get("image"):
            umap_html += f'<img src="{s["image"]}" width="250"><br>'
        if s.get("website"):
            umap_html += f'<a href="{s["website"]}" target="_blank">🌍 Website</a><br>'
        if s.get("osm_rel"):
            umap_html += f'<a href="https://www.openstreetmap.org/relation/{s["osm_rel"]}" target="_blank">🗺 OSM Relation</a><br>'
        umap_html += f'<a href="{s["wikidata"]}" target="_blank">🔗 Wikidata</a>'

        props["UmapPop"] = umap_html

        feature = Feature(geometry=mapping(Point(s["coord"])), properties=props)
        features.append(feature)
    return FeatureCollection(features)

# Run and export
if __name__ == "__main__":
    print("Fetching SPARQL point data...")
    section_data = run_sparql(sparql_query)
    print(f"Found {len(section_data)} trail sections with P625 coordinates.")

    print("Building GeoJSON from coordinates...")
    geojson = build_geojson_points(section_data)

    timestamp = datetime.datetime.now().strftime("%Y%m%d%H_%M_%S")
    output_path = f"../geojson/Issue_2_P625_points_{timestamp}.geojson"

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(geojson, f, ensure_ascii=False, indent=2)
    print(f"✅ GeoJSON file created: {output_path}")


Fetching SPARQL point data...
Found 11 trail sections with P625 coordinates.
Building GeoJSON from coordinates...
✅ GeoJSON file created: ../geojson/Issue_2_P625_points_2025072719_07_24.geojson


In [4]:
    end_time = time.time()
    duration = end_time - start_time
    print(f"GeoJSON saved to {output_path}")
    print(f"Finished in {duration:.2f} seconds.")


GeoJSON saved to ../geojson/Issue_2_P625_points_2025072719_07_24.geojson
Finished in 1.09 seconds.
